In [ ]:
num_samples = 1000
num_channels = 8
samp_freq = 300
seq_len = 2200
label_names = ['1dAVb', 'RBBB', 'LBBB', 'SB', 'AF', 'ST']

In [ ]:
import pickle as pkl

with open("/data/train.pkl", 'rb') as file:
    train = pkl.load(file)

with open("/data/dev.pkl", 'rb') as file:
    dev = pkl.load(file)

train.normalize()
dev.normalize()

In [ ]:
from tensorflow.keras import models, layers

inputs = (seq_len, num_channels)
num_labels = len(label_names)

model = models.Sequential([
    layers.Input(inputs),
    layers.Conv1D(filters=filter, kernel_size=2, activation='relu'),
    layers.MaxPooling1D(2),
    layers.Conv1D(filters=2*filter, kernel_size=2, activation='relu'),
    layers.MaxPooling1D(2),
    layers.Conv1D(filters=2*(2*filter), kernel_size=2, activation='relu'),
    layers.MaxPooling1D(2),
    layers.Flatten(),
    layers.Dropout(0.5),
    layers.Dense(2*(2*filter), activation='relu'),
    # Sigmoid for multi-label classification
    layers.Dense(num_labels, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', 
                    optimizer='adam', 
                    metrics=['accuracy'])

In [ ]:
model.fit(train.X, train.y, epochs=10, batch_size=16, validation_data=(dev.X, dev.y))

_, acc = model.evaluate(dev)
print(f"Model accuracy: {acc*100:.2f}%")